# CLIENT (SENDER)

In [1]:
import numpy as np
import binascii

In [2]:
def decimalToBinary(num , places):
    return ('{0:0' +  str(places) + 'b}').format(num)


def binaryToDecimal(binary):
    return int(binary, 2)
        
def xorBinary(binary,key = "00000000"):
    i=0
    encBin = ""
    while(i<8):
        if(binary[i] == '0' and key[i] == '0') or (binary[i] == '1' and key[i] == '1'):
            encBin+="0"
        else:
            encBin+="1"
        i+=1
        
#     print(binary , key , encBin)    
    return encBin
    
def compressBinary(binary):
    i=0
    compressBin = ""
    while(i<8):
        if(i%2 == 0):
            if((binary[i] == '0' and binary[i+1] == '0') or (binary[i] == '1' and binary[i+1] == '1')):
                compressBin+="0"
            else:
                compressBin+="1"
        i+=1
        
    return compressBin

def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)

# Code for importing data

In [3]:
import pandas as pd
article_read = pd.read_csv('dataset.csv',delimiter = ',')

In [4]:
# new_dataset = pd.Series(list(article_read['Temp.'].sample(101 , random_state=10)))
new_dataset = article_read['Temp.']

# new_dataset

# Code for delta coding

In [7]:
new_dataset.describe()

count    928991.000000
mean         27.283373
std           0.904537
min          24.434400
25%          26.764900
50%          27.328400
75%          27.868700
max          30.325400
Name: Temp., dtype: float64

In [8]:
int_dataset = new_dataset * 10000

In [9]:
int_dataset = int_dataset.astype('int64')

In [10]:
int_dataset.head()

0    262257
1    262308
2    262365
3    262416
4    262462
Name: Temp., dtype: int64

In [11]:
diff_dataset = [int_dataset[0]]
for i in range(1 , len(int_dataset)) :
        diff_dataset.append(int_dataset[i] - int_dataset[i-1])

In [12]:
diff_dataset = pd.Series(diff_dataset)

In [13]:
diff_dataset.head()

0    262257
1        51
2        57
3        51
4        46
dtype: int64

In [14]:
import seaborn as sns
%matplotlib inline

In [15]:
diff_dataset.describe()

count    928991.000000
mean          0.298587
std         295.352490
min      -41416.000000
25%          -7.000000
50%           0.000000
75%           6.000000
max      262257.000000
dtype: float64

In [16]:
len(diff_dataset)

928991

In [17]:
((diff_dataset > 64) | (diff_dataset < -64)).sum() 

24877

In [18]:
24877/928991 * 100

2.6778515615328886

In [19]:
(2.6778515615328886 * 24 + (100 - 2.6778515615328886) *  8) / (100 * 20)


0.4214228124922631

In [36]:
bytesToSend = []
for i in diff_dataset :
    if(i > 64 or i < -64) :
        # Send 3 Bytes
        number = decimalToBinary(abs(i) , 20)
        if(i < 0):
            firstByte = '01' + number[0:6]
        else :
            firstByte = '00' + number[0:6]
        secondByte = '0' + number[6:13]
        thirdByte = '0' + number[13:]
        bytesToSend.append(firstByte)
        bytesToSend.append(secondByte)
        bytesToSend.append(thirdByte)
    else :
        number = decimalToBinary(abs(i) , 6)
        if(i < 0):
            firstByte = '11' + number
        else :
            firstByte = '10' + number
        bytesToSend.append(firstByte)

# Code for Stream

In [21]:
import time

In [22]:
import socket
# host = 'localhost'  # as both code is running on same pc

host = "192.168.43.136"
port = 5000  # socket server port number

message_array = []

In [23]:
def sendMessage(dataByte , socket):
    socket.send(dataByte)

In [24]:
def storeMessage(dataByte):
    message_array.append(dataByte)

In [47]:
def client_program(obv_factor):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # instantiate
    client_socket.connect((host, port))  # connect to the server

    count = 0
    for i in range(0, int(len(bytesToSend) * obv_factor)):
        final = binaryToDecimal(bytesToSend[i]).to_bytes(1, byteorder="little")
        storeMessage(final)
        count += 1

    for i in message_array :
        sendMessage(i , client_socket)

    print("Packets Transferred : " , count)    
    client_socket.send(''.encode())
    client_socket.close()  # close the connection

# Code for Un-compressed

In [45]:
UCbytesToSend = []
for i in diff_dataset :
    number = decimalToBinary(abs(i) , 23)
    if(i < 0):
        firstByte = '1' + number[0:7]
    else :
        firstByte = '0' + number[0:7]
    secondByte = number[7:15]
    thirdByte = number[15:]
    UCbytesToSend.append(firstByte)
    UCbytesToSend.append(secondByte)
    UCbytesToSend.append(thirdByte)

In [49]:
def UC_client_program(obv_factor):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # instantiate
    client_socket.connect((host, port))  # connect to the server

    count = 0
    for i in range(0, int(len(UCbytesToSend) * obv_factor)):
        final = binaryToDecimal(UCbytesToSend[i]).to_bytes(1, byteorder="little")
        storeMessage(final)
        count += 1

    for i in message_array :
        sendMessage(i , client_socket)

    print("Packets Transferred : " , count)    
    client_socket.send(''.encode())
    client_socket.close()  # close the connection

In [50]:
len(UCbytesToSend)

2786973